# Introduction

The aim of this project is to get a tool to find neighbourhoods with specific characteristics. When people move on to a different city there usually are some essencial venues and services they want on the new neighbourhood. For example some people may want to have a train station, or scholls near their new home or, maybe, the only want to find a neighbourhood with similar characteristics than they actual neighbourhood. In this project we will try the Foursqare Api to explore the city of Toronto in order to find the more situable neigbourhood for someone moving from an specific place.

# Data

The data used to investigate Toronto neighbourhood is extracted from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

The data was extracted form the webpage by usint BeautifulSoup library

In [3]:
pip install bs4

     |████████████████████████████████| 122kB 6.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4
Note: you may need to restart the kernel to use updated packages.


In [3]:
# library for BeautifulSoup, for web scrapping
from bs4 import BeautifulSoup
# library to handle requests
import requests
import pandas as pd
import numpy as np

wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_page = requests.get(wikipedia_link)

# Cleans html file
soup = BeautifulSoup(wikipedia_page.content, 'html.parser')
# This extracts the "tbody" within the table where class is "wikitable sortable"
table = soup.find('table', {'class':'wikitable sortable'}).tbody
# Extracts all "tr" (table rows) within the table above
rows = table.find_all('tr')
# Extracts the column headers, removes and replaces possible '\n' with space for the "th" tag
columns = [i.text.replace('\n', '')
           for i in rows[0].find_all('th')]
# Converts columns to pd dataframe
df = pd.DataFrame(columns = columns)


In [5]:
#Extracts every row with corresponding columns then appends the values to the create pd dataframe "df". The first row (row[0]) is skipped because it is already the header

for i in range(1, len(rows)):
    tds = rows[i].find_all('td')    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df = df.append(pd.Series(values, index = columns), ignore_index = True)
        df

The final data frame is showed bellow

In [6]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Lets delete not assigned boroughs in order to use the data

In [9]:
toronto_df=df.drop(df[df.Borough == 'Not assigned'].index)
toronto_df = toronto_df.reset_index(drop=True)
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Longitude and latitude for every Neighbourhood

In order to get the Foursquare API information lets get the lotitude and latitude information from every nighbourhood

To do this I will use Geocoder library

In [19]:
!pip -q install geopy
from geopy.geocoders import Nominatim

# install the Geocoder
!pip -q install geocoder
import geocoder

# import time
import time

In [20]:
# Geocoder starts here
# Defining a function to use --> get_latlng()'''
def get_latlng(arcgis_geocoder):
    
    # Initialize the Location (lat. and long.) to "None"
    lat_lng_coords = None
    
    # While loop helps to create a continous run until all the location coordinates are geocoded
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [22]:
start = time.time()
postal_codes = toronto_df['Postal Code']    
coordinates = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]
end = time.time()
print("Time of execution: ", end - start, "seconds")

Time of execution:  59.82278513908386 seconds


In [23]:
toronto_df_loc = toronto_df

In [26]:
# The obtained coordinates (latitude and longitude) are joined with the dataframe as shown
toronto_df_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
toronto_df_loc['Latitude'] = toronto_df_coordinates['Latitude']
toronto_df_loc['Longitude'] = toronto_df_coordinates['Longitude']
toronto_df_loc.head(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75188,-79.33036
1,M4A,North York,Victoria Village,43.73042,-79.31282
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65514,-79.36265
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72321,-79.45141
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66449,-79.39302


## Foursquare API

In [27]:
CLIENT_ID = 'YSJNVPY5QDYMNVGV4CLQSYSODUYDTQ54SGTFR5ZWUUTAWF0B' # your Foursquare ID
CLIENT_SECRET = 'RRKMWVYNK3RZOT0YYPF3DANVOQ2Z1RT14PGUZRBB0BHXN4EH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YSJNVPY5QDYMNVGV4CLQSYSODUYDTQ54SGTFR5ZWUUTAWF0B
CLIENT_SECRET:RRKMWVYNK3RZOT0YYPF3DANVOQ2Z1RT14PGUZRBB0BHXN4EH


#### Function to obtain venues for every neighbour

In [28]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get the venues

In [30]:
toronto_venues = getNearbyVenues(names=toronto_df_loc['Neighbourhood'],
                                   latitudes=toronto_df_loc['Latitude'],
                                   longitudes=toronto_df_loc['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

Now we have the information to investigate the Neighbourhood in Toronto, look for specific services and compare them with other neighbourhood in different cities.